In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

import os, sys
sys.path.append('../code')

from pomegranate.distributions import Categorical
from pomegranate.hmm import DenseHMM

from duree import duree   # on l'utilisera plus tard pour les durées

In [5]:
# paramètres symboliques de la chaîne
alpha = 0.9   # P(pluie demain | pluie aujourd'hui)
beta  = 0.3   # P(pluie demain | sec aujourd'hui)
gamma = 0.5   # P(sec au temps 0)

A = np.array([[1-beta, beta],
              [1-alpha, alpha]])

pi0 = np.array([gamma, 1-gamma])

print("Matrice de transition A =\n", A)
print("Distribution initiale pi0 =", pi0)

Matrice de transition A =
 [[0.7 0.3]
 [0.1 0.9]]
Distribution initiale pi0 = [0.5 0.5]


In [6]:
# solution théorique de l'équation stationnaire
pi1_star_theo = beta / (1 + beta - alpha)
pi0_star_theo = 1 - pi1_star_theo
pi_star_theo = np.array([pi0_star_theo, pi1_star_theo])

print("pi* théorique =", pi_star_theo)

# vérification que pi* = pi* A
print("pi* A =", pi_star_theo @ A)

pi* théorique = [0.25 0.75]
pi* A = [0.25 0.75]


In [7]:
Nsamples = 100000

# Distributions d'émission (identité)
d0 = Categorical([[1.0, 0.0]])
d1 = Categorical([[0.0, 1.0]])
distributions = [d0, d1]

edges = A.tolist()
starts = pi0.tolist()
ends   = [0.0, 0.0]

model = DenseHMM(
    distributions,
    edges=edges,
    starts=starts,
    ends=ends,
    sample_length=Nsamples,
    return_samples=True,
    return_sample_paths=True
)

# Génération d'une séquence
obsSeq, statesSeq = model.sample(1)   # 1 séquence de longueur Nsamples
obsSeq = obsSeq[0][:, 0]              # observations 0/1
statesSeq = statesSeq[0]              # états cachés (ici identiques aux obs)

# Fréquence empirique de pluie
freq_pluie_emp = np.mean(obsSeq == 1)

print("Fréquence empirique de pluie :", freq_pluie_emp)
print("Probabilité stationnaire théorique :", pi1_star_theo)
print("Écart absolu :", abs(freq_pluie_emp - pi1_star_theo))

TypeError: DenseHMM.__init__() got an unexpected keyword argument 'return_samples'. Did you mean 'return_sample_paths'?